## Mapping ecoregions and the ecoregional zone to which the stations belong 
### This notebook will output the ecoregions (province) of the input station list.

The ecoregions file are from United States Environmental Protection Agency.<br>
You can find data downloading in this URL:
https://www.epa.gov/eco-research/ecoregions-north-america <br>
(The example folder has the file downloaded already).

In [ ]:
#!pip install fiona dbfread

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import fiona
import pygmt

In [ ]:
# Path to the shapefile
file_path = "NA_CEC_Eco_Level3/NA_CEC_Eco_Level3.shp"

# Try to read the shapefile using Fiona
try:
    with fiona.open(file_path) as src:
        # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(src, crs=src.crs)
        print("Shapefile loaded successfully.")

        # Plot the GeoDataFrame
        fig, ax = plt.subplots(figsize=(10, 10))
        gdf.plot(ax=ax, color="lightgreen", edgecolor="black", label="Washington Geology")
        ax.set_title("Structural Geology of Washington")
        ax.set_aspect('equal')
        ax.legend()
        plt.show()
except Exception as e:
    print(f"Error loading shapefile: {e}")


In [ ]:
# Check the contents of the GeoDataFrame
print(gdf.columns)

In [ ]:
from dbfread import DBF

# Path to the .dbf file
dbf_path = "NA_CEC_Eco_Level3/NA_CEC_Eco_Level3.dbf"

# Read the .dbf file into a DataFrame
table = DBF(dbf_path)
df = pd.DataFrame(iter(table))

# Display the contents of the DataFrame
df.head()


In [ ]:
import geopandas as gpd

# Path to the shapefile
shp_path = "NA_CEC_Eco_Level3/NA_CEC_Eco_Level3.shp"

# Load the shapefile
gdf = gpd.read_file(shp_path)

# Check the contents of the GeoDataFrame
print(gdf.head())


In [ ]:
# Initialize the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the merged GeoDataFrame
# Replace 'geological_type' with the actual column name you want to visualize
gdf.plot(ax=ax, column='NA_L1CODE',  # Change to your actual column name
                cmap='Set3',  # Choose a color map
                edgecolor='black',  # Outline color for each shape
                legend=True)  # Show legend

# Add title and labels
ax.set_title("Ecoregions Level-1 Map of North America")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_aspect('equal')  # Maintain aspect ratio

# Show the plot
plt.show()


In [ ]:

# Initialize the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the merged GeoDataFrame
# Replace 'geological_type' with the actual column name you want to visualize
gdf.plot(ax=ax, column='NA_L2CODE',  # Change to your actual column name
                cmap='Set3',  # Choose a color map
                edgecolor='black',  # Outline color for each shape
                legend=True)  # Show legend

# Add title and labels
ax.set_title("Ecoregions Level-2 Map of North America")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_aspect('equal')  # Maintain aspect ratio

# Show the plot
plt.show()


In [ ]:

# Initialize the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the merged GeoDataFrame
# Replace 'geological_type' with the actual column name you want to visualize
gdf.plot(ax=ax, column='NA_L3CODE',  # Change to your actual column name
                cmap='Set3',  # Choose a color map
                edgecolor='black',  # Outline color for each shape
                legend=True)  # Show legend

# Add title and labels
ax.set_title("Ecoregions Level-3 Map of North America")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_aspect('equal')  # Maintain aspect ratio

# Show the plot
plt.show()


In [ ]:
gdf[[ 'NA_L3CODE', 'NA_L3NAME', 'geometry']]

Convert the coordination to WGS84. 

In [ ]:
# Step 2: Convert to EPSG:4326 (longitude/latitude)
geology_gdf = gdf.to_crs(epsg=4326)
geology_gdf[[ 'NA_L3CODE', 'NA_L3NAME', 'geometry']]

In [ ]:
# Load the station list with latitude and longitude
stations_path = "CI.csv"  # Replace with actual file path
stations_df = pd.read_csv(stations_path)
print(stations_df.head())

# Convert the station DataFrame to a GeoDataFrame
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df.stlo, stations_df.stla),
    crs="EPSG:4326"  # Assuming coordinates are in WGS 84
)
print(stations_gdf.head())
# Perform spatial join between stations and geological units
station_geology_gdf = gpd.sjoin(stations_gdf, geology_gdf, how="inner", predicate="within")

# Check results
print(station_geology_gdf[['netst', 'NA_L3CODE']])  # Replace with actual columns

station_geology_gdf.to_csv('ca_station_ecoregion.csv', index=False)

In [ ]:
# Load CSV data
eco_df=pd.read_csv('ca_station_ecoregion.csv')

In [ ]:
from matplotlib.colors import to_hex

df = eco_df

region=[-122,-114,32,36]
proj='M6c'
# Initialize the figure
fig = pygmt.Figure()
pygmt.config(MAP_GRID_PEN = '0.01p,150' , MAP_FRAME_PEN='0.05p', MAP_FRAME_TYPE="plain", MAP_TITLE_OFFSET="0.12p", FONT_TITLE="12p", FONT_ANNOT='4p',  )
 
# Plot the basemap
fig.basemap(region=region,  projection=proj, frame="a2g1", )
grid = pygmt.datasets.load_earth_relief(resolution="30s", region=region)
fig.grdimage(grid='@earth_day_30s', projection=proj, transparency=60, )

fig.coast(region=region, projection=proj, frame="a2g1", resolution="f", borders="2/0.05p",
            water='lightblue', shorelines='0/0.1p',)
# Plot each unique NA_L3KEY as a different color
unique_keys = df['NA_L3NAME'].unique()

# Get colormap
cmap = plt.get_cmap('Set3', len(unique_keys))
# Plot each unique NA_L3NAME with corresponding color from the colormap
for i, key in enumerate(unique_keys):
    subset = df[df['NA_L3NAME'] == key]
    
    # Check if subset is not empty
    if not subset.empty and len(subset) > 0:
        color = to_hex(cmap(i)[:3])  # Get the color from the colormap
        print(f"Plotting {key} with color {color}")
        # Plot with the same length for x and y
        fig.plot(
            x=subset['stlo'],
            y=subset['stla'],
            style="s0.1c",  # Circle style with size
            pen="0.1p,black",
            fill=color,  # Use RGB values; color is returned as (R, G, B, A)
            label=key
        )
    else:
        print(f"No data to plot for {key}")

# Add a legend
fig.legend(position="JMR", )
fig.show(dpi=600)
